# Inner function (nested function)
함수 안에서 함수 만들기

In [1]:
# def 함수이름1():
#     코드
#     def 함수이름2():
#         코드

In [4]:
def print_hello():
    hello = "Hello, world!"
    
    # ↓함수안에서 정의된 함수 (inner function 혹은 nested function)
    def print_message():  
        print(hello)   # 안쪽 scope 에서 바깥쪽 scope의 변수 사용 '읽기' 가능
        
    print_message()
    
print_hello()

# print_message()  # inner function 을 함수 밖에서 원칙적으로 사용 불가

Hello, world!


In [ ]:
# 함수를 리턴하는 함수
# 곱하는 함수를 만드는 함수

In [5]:
def multiplexer(x):
    def something(n):
        return x * n
    
    return something   # inner function 자체를 리턴!

multiplexer(3)

<function __main__.multiplexer.<locals>.something(n)>

In [6]:
triple = multiplexer(3)
double = multiplexer(2)

# triple과 double 은 함수다!

In [10]:
triple(100), triple(10), triple(1000)

(300, 30, 3000)

In [12]:
double(100), double(10), double(1000)

(200, 20, 2000)

#### 위 코드를 lamda function 으로 변경!

In [13]:
multiplexer = lambda x : lambda n : x * n

triple = multiplexer(3)
double = multiplexer(2)

triple(100), double(100)

(300, 200)

---
## inner function  에서 scope 문제

#### 외부 scope 의 변수 '읽기'

In [14]:
def greetings(x):
    def say_hi():
        print(x)  # x 는 say_hi() 입장에선 외부 scope의 변수다. '읽기 가능'
        
    say_hi()
    
greetings('안녕하세요?')

안녕하세요?


#### 외부 scope 의 변수 '변경'  (쓰기)

In [15]:
def A():
    x = 10      # A 의 지역변수 x
    def B():
        x = 20  # inner local 에선 outer local의 변수 변경(쓰기) 원칙적으로 불가
                # 따라서 x 는 B의 지역변수 x
    B()
    print(x)    # A 의 지역변수 x
    
A()

10


In [16]:
def count(x):
    def increment():
        x = x + 1
        print(x)
        
    increment()
    
count(5)

# UnboundLocalError: local variable 'x' referenced before assignment

UnboundLocalError: local variable 'x' referenced before assignment

#### Quiz

In [19]:
# Quiz : 화면에는 무엇이 찍히겠는가
# 실행하기 전에 맞추어 보자.

z = 3

def outer(x):
    y = 10
    def inner():
        x = 1000
        return x

    return inner()


print(outer(1))
print(outer(10))
print(outer(100))

1000
1000
1000


## nonlocal
현재 함수의 바깥쪽에 있는 지역변수의 값을 '변경' 하려면 nonlocal 키워드 사용해야 한다

In [21]:
def A():
    x = 10    
    def B():
        nonlocal x   # x는 B 의 local 이 아닌 바깥족의 local 을 사용
        x = 20 
               
    B()
    print(x)   
    
A()

20


#### nonlocal 이 변수를 찾는 순서

In [24]:
def A():
    x = 10
    y = 100
    def B():
        x = 20
        def C():
            nonlocal x
            nonlocal y
            x = x + 30
            y = y + 300
            print('C() x =', x)
            print('C() y =', y)
            
        C() 
        print('B() x =', x)
        print('B() y =', y)        
    B()
    print('A() x =', x)
    print('A() y =', y)        
A()
    

C() x = 50
C() y = 400
B() x = 50
B() y = 400
A() x = 10
A() y = 400


### global 전역변수 사용
inner function이 몇단계이든 상관없이 global 키워드는 무조건 전역변수 사용!

In [27]:
x = 1
def A():
    x = 10
    def B():
        x = 20
        def C():
#             nonlocal x
            global x
            x = x + 30
            print(x)
        C()
    B()
A()

print(x)


31
31


In [28]:
# 파이썬에서 global을 제공하지만 
# 함수에서 값을 주고받을 때는 매개변수와 반환값을 사용하는 것이 좋습니다. 
# 특히 전역 변수는 코드가 복잡해졌을 때 변수의 값을 어디서 바꾸는지 알기가 힘듭니다. 
# 따라서 전역 변수는 가급적이면 사용하지 않는 것을 권장합니다.

---
# Closure (클로저)

In [32]:
def calc():
    a = 3
    b = 5
    
    def mul_add(x):
        return a * x + b  # 함수 바깥쪽에 있는 지역변수 a, b 를 사용하여 계산
    
    return mul_add   # 내부에서 만든 mul_add 함수를 리턴

c = calc()
    
print(c(1), c(2), c(3), c(4), c(5))

# print(a)

8 11 14 17 20


## closure 란?
잘 보면 함수 calc가 끝났는데도 c는 calc의 지역 변수 a, b를 사용해서 계산을 하고 있습니다. <br>
`이와같이 함수를 둘러싼 환경(지역 변수, 코드 등)을 계속 유지하다가, <br>
함수를 호출할 때 다시 꺼내서 사용하는 함수를 **클로저(closure)**라고 합니다. `<br>
여기서는 c에 저장된 함수가 클로저입니다.

**‘자신을 둘러싼 스코프(네임스페이스)의 상태값을 기억하는 함수’**

![](https://dojang.io/pluginfile.php/13868/mod_page/content/3/033004.png)


#### 어떤 함수가 클로저 이기 위한 조건
1. 해당 함수는 어떤 함수 내의 중첩된 함수여야 한다.
1. 해당 함수는 자신을 둘러싼(enclose) 함수 내의 상태값을 반드시 참조해야 한다.
1. 해당 함수를 둘러싼 함수는 이 함수를 반환해야 한다.

### lambda 로 클로저 만들기
람다를 사용하면 클로저를 좀 더 간편하게 만들 수 있습니다.

In [33]:
def calc():
    a = 3
    b = 5
    return lambda x : a * x + b   # 람다표현식을 리턴

c = calc()
print(c(1), c(2), c(3), c(4), c(5))

8 11 14 17 20


### 클로저에서 지역변수 변경(수정)하기
클로저의 지역변수를 변경하고 싶다면 **nonlocal** 을 사용하면 된다.

In [39]:
def calc():
    a = 3 
    b = 5
    total = 0
    
    def mul_add(x):
        nonlocal total
        print('prev total', total)
        total = total + a * x + b
        print('after total', total)
        
    return mul_add

c = calc()
c(1)
c(2)
c(3)

prev total 0
after total 8
prev total 8
after total 19
prev total 19
after total 33


# 연습 : 호출 횟수를 세는 함수 만들기

In [40]:
def counter():
    i = 0
    def count():
        nonlocal i;
        i += 1
        return i
        
    return count;

c = counter()
for i in range(10):
    print(c(), end=' ')
    
# [결과]    
# 1 2 3 4 5 6 7 8 9 10 

1 2 3 4 5 6 7 8 9 10 

## 클로저의 사용이유
1. 관리와 책임을 명확히 할 수 있고
1. 각 변수가 섞여 불필요한 충돌을 방지할 수 있으며
1. 사용환경(context)에 맞게 임의대로 내부구조를 조정할 수 있다.